# Before begins

This notebook is written in google colab.

To see some interactive plots, please enter the colab link Below.

<a href="https://colab.research.google.com/drive/1FihAHMXlpPxLwlpa-B261IcY1zisMrEL?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Overview

<br>

## Competition description

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/3948/logos/thumb76_76.png" width=50 align='left' alt="Open in Colab"/></a>
&nbsp; 
<font size="5">[Bike Sharing Demand](https://www.kaggle.com/c/house-prices-advanced-regression-techniques)</font>

<br>

- Problem type: Multi-output regression
  - Forecast demand of a city bikeshare system
- Evaluation metric: Root Mean Squared Log Error (RMSLE)

<br>

## Notebook description

This notebook provides the '**proper workflow**' for kaggle submission.

The workflow is divided into three main steps.
1. Data preprocessing
2. Model selection (hyper parameter tuning, model combination, model comparison)
3. Training final model & Prediction on Test-set

At each stage, detailed descriptions of the work and an appropriate procedure will be provided.

Through this notebook, readers can learn the 'proper workflow' to be done for kaggle submission, 
and using this as a basic structure, someone will be able to apply this to other competitions easily with some adjustments

**Warnings**:
- The purpose of this notebook
  - This notebook focuses on the 'procedure' rather than the 'result'. 
  - Thus this notebook does not guide you on how to achieve the top score. Since I personally think that any result can only have a meaning through an appropriate procedure.
  - But since this is a competition, it cannot be avoided that the score is important. Following this notebook, you will get the top 15% (score: 0.12519) result in this competition

- The readers this notebook is intended for
  - Who are aware of the basic usage of data processing tools (e.g., numpy, pandas)
  - Who are aware of the basic concepts of machine learning models 


# 0. Preliminaries

### > Set Configurations 

- Set the configurations for this notebook

In [ ]:
config = {
    'data_name': 'Bike_Sharing_Demand',
    'random_state': 2022
}

### > Install Libraries

In [ ]:
!pip install tune_sklearn ray[tune] skorch

# 1. Data preprocessing

The data preprocessing works are divided into 9 steps here.

Some of these steps are mandatory and some are optional.

Optional steps are marked separately.

It is important to go through each step in order.
Be careful not to reverse the order.

## 1-1. Load Dataset

Load train-set and test-set on working environment


### > Download Data from Kaggle by using Kaggle API

Navigate to https://www.kaggle.com. Then go to the [Account tab of your user profile](https://www.kaggle.com/me/account) and select Create API Token. This will trigger the download of kaggle.json, a file containing your API credentials.

Then run the cell below to upload kaggle.json to your Colab runtime.

In [ ]:
from google.colab import files

# Upload Kaggle API key (kaggle.json)
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
%%bash 

(mkdir Bike_Sharing_Demand
cd Bike_Sharing_Demand
kaggle competitions download -c bike-sharing-demand
)

In [ ]:
import numpy as np
import pandas as pd

train = pd.read_csv('/content/{}/train.csv'.format(config['data_name']))
test = pd.read_csv('/content/{}/test.csv'.format(config['data_name']))

### > Concatenate the 'train' and 'test' data for preprocessing

Data preprocessing work should be applied equally for train-set and test-set.

In order to work at once, exclude the output variables ['casual', 'registered', 'count'] from 'train' and combine it with 'test'.

In [ ]:
all_features = pd.concat([train.drop(['casual', 'registered', 'count'], axis=1), test], axis=0)
y_train = train[['casual', 'registered', 'count']]

## 1-2. Missing Value Treatment

Missing (NA) values in Data must be treated properly before model training.

There are three main treatment methods:
1. Remove the variables which have NA values
2. Remove the rows (observations) which have NA values
3. Impute the NA values with other values

Which of the above methods is chosen depends on the analyst's discretion.
It is important to choose the appropriate method for the situation.

### > Check missing values in each variable

There is no missing values in the data-set

In [ ]:
all_features.isnull().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
dtype: int64

## 1-3. Variable modification

### > datetime variable

From 'datetime' variable, we can extract the following informations 
- year
- month 
- day
- hour
- name_of_day

In [ ]:
all_features.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0


In [ ]:
all_features['datetime'] = pd.to_datetime(all_features['datetime'])

all_features['year'] = all_features['datetime'].dt.year
all_features['month'] = all_features['datetime'].dt.month
all_features['day'] = all_features['datetime'].dt.day
all_features['hour'] = all_features['datetime'].dt.hour
all_features['day_name'] = all_features['datetime'].dt.day_name()

all_features.drop(['datetime'], axis=1, inplace=True)

### > Drop unuseful variables

#### >> 'day'

In this competition, 1st to the 19th day of each month are divided into train data, and from the 20th to the end of the month are divided into test data.
Thus 'day' variable which indicates the day of the bike rental does not provide any sufficient information in training and prediction




In [ ]:
all_features.drop(['day'], axis=1, inplace=True)

## 1-4. Variable transformation

### > int to object

In [ ]:
# int -> object
vars = ['season', 'holiday', 'workingday', 'weather', 'year', 'month', 'hour']
all_features[vars] = all_features[vars].astype('object')

## 1-5. Dummify categorical variables

In the case of linear modeling without regularization, the first or last column should be dropped (to prevent linear dependency), but here, for the convenience of using the factorization model, one-hot encoding method is used that does not drop any columns.

In [ ]:
data_set = pd.get_dummies(all_features, drop_first=False)

## 1-6. Scaling continuous variables


MinMaxScaling maps all variables from 0 to 1 in order to consider only relative information, not absolute magnitudes of the values.

Besides, it is known that scaling is often more stable in parameter optimization when training a model.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_set = scaler.fit_transform(data_set)

## 1-7. Split Train & Test set

In [ ]:
n_train = train.shape[0]
X_train = data_set[:n_train].astype(np.float32)
X_test = data_set[n_train:].astype(np.float32)
y_train = y_train.astype(np.float32)

## 1-8. Outlier Detection (*optional*)

Detect and remove outlier observations that exist in the train-set.

- Methodology: [Isolation Forest](https://ieeexplore.ieee.org/abstract/document/4781136/?casa_token=V7U3M1UIykoAAAAA:kww9pojtMeJtXaBcNmw0eVlJaXEGGICi1ogmeHUFMpgJ2h_XCbSd2yBU5mRgd7zEJrXZ01z2)
  - How it works
    - Isolation Forest applies a decision tree that repeats splits based on the 'random criterion' for the given data unitl only one observation remains in every terminal node (this is defined as 'isolation').
    - Based on the number of splits used for isolation, 'normality' is defined. A smaller value means a higher degree of outlierness.
    - By applying this decision tree several times, the average of the measured 'normality' values ​​is derived as the final 'normality' value.
  - Assumptions
    - Outliers require relatively few splits to be isolated.
    - For normal data, the number of splits required to be isolated is relatively large.
  - Outlier determination
    - Determines whether it is an outlier or not based on the measured 'normality' value.
      - sklearn's IsolationForest package determines based on '0' 
      - I, personally, think it is better to set the discriminant criterion by considering the 'distribution' of the 'normality' values.
      - The details of the method is given below.

In [ ]:
from sklearn.ensemble import IsolationForest
clf = IsolationForest(
    n_estimators=100,
    max_samples='auto',
    n_jobs=-1,
    random_state=config['random_state'])

clf.fit(X_train)
normality_df = pd.DataFrame(clf.decision_function(X_train), columns=['normality'])

- The dicriminant value 
  - The discriminant value (threshold) is defined by calculating the 1st quartile ($q_1$) and 3rd quartile ($q_3$) on the distribution of the measured normality values.
    - with $k=1.5$

$$threshold = q_1 - k*(q_3 - q_1)$$


- Motivation
  - This discriminant method is adapted from Tukey's boxplot idea.
In the distribution of any continuous variable, Tukey designates observations smaller than that value or larger than q_3 + k*(q_3 - q_1) as outliers.

- How we do 
  - Our methodology does not apply the above method to a specific variable, but applies the method to the obtained normality.

  - That is, it is based on the assumption that an outlier will be far left from the other observations in the measured normality distribution.

In [ ]:
def outlier_threshold(normality, k=1.5):
  q1 = np.quantile(normality, 0.25)
  q3 = np.quantile(normality, 0.75)  
  threshold = q1 - k*(q3-q1)
  return threshold

threshold = outlier_threshold(normality_df['normality'].values, k=1.5)

import plotly.express as px
fig = px.histogram(normality_df, x='normality', width=400, height=400)
fig.add_vline(x=threshold, line_width=3, line_dash="dash", line_color="red")
fig.show()

In [ ]:
import plotly.express as px
px.box(normality_df, x='normality', orientation='h', width=400, height=400)

In [ ]:
X_train = X_train[normality_df['normality'].values>=threshold]
y_train = y_train[normality_df['normality'].values>=threshold]

print('{} out of {} observations are removed from train_set'.format(train.shape[0] - X_train.shape[0], train.shape[0]))

346 out of 10886 observations are removed from train_set


## 1-9. Output variable consideration



### > Multi-output variables

According to the data description, the output variable 'count' is divided into two variables 'casual' and 'registered'. (i.e., count = registered + casual)

#### >> Distribution differences between 'registerd' and 'casual'

- Along 'hour' 
  - From the figure below, we can observe that the distributions between 'registered' and 'casual' are different along the 'hour'.
  - 'registered' are more likely to rent bikes at commute time and 'casual' are more likely to rent bikes at daytime

In [ ]:
df = pd.concat([all_features[:train.shape[0]], train[['casual', 'registered', 'count']]], axis=1)

df_melt = pd.melt(df, id_vars=['hour'], value_vars=['casual', 'registered'])

fig = px.box(df_melt, x='hour', y='value', color='variable', width=600)
fig.show()

- By 'workingday'
  - From the figure below, we can observe that 'registered' tend to rent bikes on working days on the contrary 'casual' are more likely to rent bikes on not working days

In [ ]:
df = pd.concat([all_features[:train.shape[0]], train[['casual', 'registered', 'count']]], axis=1)

df_melt = pd.melt(df, id_vars=['workingday'], value_vars=['casual', 'registered'])

fig = px.box(df_melt, x='workingday', y='value', color='variable', width=600)
fig.show()

Thus intead of 'count', 'casual' and 'registered' are obtained as our output variables.

At the final prediction, the predicted values for 'casual' and 'registered' will be sumed up. 

### > Output variable transformation

- Motivation
  - Since this is a 'count regression', whose target variable indicates count and does not contain negative values, appropriate consideration should be given.

- Statistics vs Machine learning
  - From the perspective of Statistics, generalized linear model with proper link function is recommended instead of variable transformation like log-transformation ([Do not log-transform count data](https://besjournals.onlinelibrary.wiley.com/doi/10.1111/j.2041-210X.2010.00021.x)).

  - But from the perspecitve of Machine learning, the prediction peformance is the priority. Thus some complicated models like Random Forest or Multi-layer perceptron should be considered although these models do not meet the statistical properties.

- In this notebook
  - In this notebook, log-transformation is performed on the output variables to apply a model such as a random forest.

  - For those who want to approach it from a statistical point of view, it is recommended to use sklearn's [PoissonRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.PoissonRegressor.html) model with out log-transformation.

In [ ]:
import plotly.express as px

y_train_two = y_train[['casual', 'registered']].values.astype(np.float32)

y_train_two_trans = np.log1p(y_train_two)

# 2. Model Selection

Our goal is to build a model that predicts the bike rental count (y) by summing up the values of 'registered' and 'casual' in every hour by using informations about that hour. The formula can be expressed as:

$y = g_{registerd}(x) + h_{casual}(x) + \epsilon$

This is a Multi-output regression problem. 
With some adjustments, various machine learning models can be obtained. This notebook uses the following models.
- Linear model (Poisson regression)
- Support vector machine
- Random forest
- Xgboost
- Multi-layer perceptron
- Factorization machine

However, we have to "choose" one final methodology to make predictions on the test-set.
To do so, a “fair evaluation” of the models is essential. "Fair evaluation" must satisfy the following two conditions.

1. Select best hyperparameters for each model
  - Bad hyperparmeter values could lead to the bad model performance.
2. Same evaluation method
  - If the evaluation method is not the same, comparison between models itself is impossible.

When comparing models through an evaluation method that satisfies the above two conditions,
Only then the final model can be selected.




## 2-1. Hyper parameter tuning by using Tune_SKlearn (Ray Tune)

- Package: tune_sklearn
  - This package makes it easy to apply [Ray Tune](https://docs.ray.io/en/latest/tune/index.html) to sklearn models.
  - Ray Tune is a python package that provides various hyperparameter tuning algorithms (HyperOpt, BayesianOptimization, ...).
- Tuning procedure
  - Define an appropriate search space for each model's hyperparameters.
  - 5-fold CV (Cross Validation) is performed for each specific hyper-parameter value combination of the search space by using the hyper-parameter tuning algorithm (HyperOpt)
    - Training: Training by using Scikit-Learn and Skorch packages
    - Validation: Evaluate the model using an appropriate evaluation metric
  - The hyperparameter with the highest average score of the CV result is designated as the optimal hyperparameter of the model.
    - Save this CV result and use for model comparison



### > Define a scoring function for hyper parameter tuning

Our goal is to obatin the best hyperparameter in terms of RMSLE. 

But since we perform Multi-output regression, a custom RMSLE function that computes the loss between true_y (count) and sum of 'registered' and 'casual' should be defined.

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error

def neg_rmsle_custom(y_true_two_trans, y_pred_two):
  try:
    score = np.negative(mean_squared_error(np.log1p(np.expm1(y_true_two_trans).sum(axis=1)), 
                                         np.log1p(np.expm1(np.maximum(y_pred_two, 0)).sum(axis=1)), squared=False))
  except:
    score = np.nan
  return score  

target_metric = make_scorer(neg_rmsle_custom, greater_is_better=True)

### > Make a dataframe for containing CV results

In [ ]:
model_list = []
for name in ['linear', 'svm', 'rf', 'xgb', 'mlp', 'fm']:
  model_list.append(np.full(5, name))
  
best_cv_df = pd.DataFrame({'model': np.hstack((model_list)), 'RMSLE':None, 'best_hyper_param':None})

### > Linear model

In [ ]:
from tune_sklearn import TuneSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import SGDRegressor

# Define a search space
parameters = {
    'estimator__alpha': list(np.geomspace(1e-10, 1e-6, 5)),
    'estimator__max_iter': [1000],
    'estimator__tol': [1e-5, 1e-4, 1e-3],
    'estimator__loss': ['squared_error'],  
    'estimator__random_state': [config['random_state']]
}

# Define a Multi-output regressor
base_regr = SGDRegressor()
regressor = MultiOutputRegressor(base_regr)

# Specify a hyper parameter tuning algorithm
tune_search = TuneSearchCV(
    regressor,
    parameters,
    search_optimization='hyperopt',
    n_trials=5,
    n_jobs=-1,
    scoring={'RMSLE':target_metric},
    cv=5,
    refit='RMSLE',
    verbose=1,
    random_state=config['random_state']
    )

# Run hyper parameter tuning
X = X_train
y = y_train_two_trans
tune_search.fit(X, y)

# Save the tuning results 
model_name = 'linear'

## Save the optimal hyper parmater values
best_cv_df.loc[best_cv_df['model']==model_name, 'best_hyper_param'] = str(tune_search.best_params_)

## Save the CV results
cv_df = pd.DataFrame(tune_search.cv_results_)
cv_values = cv_df.loc[tune_search.best_index_, cv_df.columns.str.startswith('split')].values
best_cv_df.loc[best_cv_df['model']==model_name, 'RMSLE'] = cv_values[:5]

# Visualize the tuning results with parallel coordinate plot
tune_result_df = pd.concat([pd.DataFrame(tune_search.cv_results_['params']), cv_df.loc[:,cv_df.columns.str.startswith('mean')] ], axis=1)
import plotly.express as px
px.parallel_coordinates(tune_result_df, color='mean_test_RMSLE')

### Support vector machine

In [ ]:
from tune_sklearn import TuneSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.multioutput import MultiOutputRegressor

# Define a search space
parameters = {
    'estimator__alpha': list(np.geomspace(1e-7, 1e-3, 3)),
    'estimator__epsilon': list(np.geomspace(1e-5, 1e-1, 3)),
    'estimator__loss': ['huber', 'epsilon_insensitive'],
    'estimator__tol': [1e-5, 1e-4, 1e-3],
    'estimator__max_iter': [1000],
    'estimator__random_state': [config['random_state']]
}


# Define a Multi-output regressor
base_regr = SGDRegressor()
regressor = MultiOutputRegressor(base_regr)

# Specify a hyper parameter tuning algorithm
tune_search = TuneSearchCV(
    regressor,
    parameters,
    search_optimization='hyperopt',
    n_trials=10,
    n_jobs=-1,
    scoring={'RMSLE':target_metric},
    cv=5,
    refit='RMSLE',
    verbose=1,
    random_state=config['random_state']
    )

# Run hyper parameter tuning
X = X_train
y = y_train_two_trans
tune_search.fit(X, y)

# Save the tuning results 
model_name = 'svm'

## Save the optimal hyper parmater values
best_cv_df.loc[best_cv_df['model']==model_name, 'best_hyper_param'] = str(tune_search.best_params_)

## Save the CV results
cv_df = pd.DataFrame(tune_search.cv_results_)
cv_values = cv_df.loc[tune_search.best_index_, cv_df.columns.str.startswith('split')].values
best_cv_df.loc[best_cv_df['model']==model_name, 'RMSLE'] = cv_values[:5]

# Visualize the tuning results with parallel coordinate plot
tune_result_df = pd.concat([pd.DataFrame(tune_search.cv_results_['params']), cv_df.loc[:,cv_df.columns.str.startswith('mean')] ], axis=1)
import plotly.express as px
px.parallel_coordinates(tune_result_df, color='mean_test_RMSLE')

### Random forest

In [ ]:
from tune_sklearn import TuneGridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

# Define a search space
parameters = {
    'estimator__n_estimators': [100, 300],
    'estimator__criterion': ['squared_error'],
    'estimator__max_depth': [25, 30, 35],
    'estimator__max_features': ['auto'],
    'estimator__random_state': [config['random_state']]
}

# Define a Multi-output regressor
base_regr = RandomForestRegressor()
regressor = MultiOutputRegressor(base_regr)

# Specify a hyper parameter tuning algorithm
tune_search = TuneSearchCV(
    regressor,
    parameters,
    search_optimization='hyperopt',
    n_trials=6,
    n_jobs=-1,
    scoring={'RMSLE':target_metric},
    cv=5,
    refit='RMSLE',
    verbose=1,
    random_state=config['random_state']
    )

# Run hyper parameter tuning
X = X_train
y = y_train_two_trans
tune_search.fit(X, y)

# Save the tuning results 
model_name = 'rf'

## Save the optimal hyper parmater values
best_cv_df.loc[best_cv_df['model']==model_name, 'best_hyper_param'] = str(tune_search.best_params_)

## Save the CV results
cv_df = pd.DataFrame(tune_search.cv_results_)
cv_values = cv_df.loc[tune_search.best_index_, cv_df.columns.str.startswith('split')].values
best_cv_df.loc[best_cv_df['model']==model_name, 'RMSLE'] = cv_values[:5]

# Visualize the tuning results with parallel coordinate plot
tune_result_df = pd.concat([pd.DataFrame(tune_search.cv_results_['params']), cv_df.loc[:,cv_df.columns.str.startswith('mean')] ], axis=1)
import plotly.express as px
px.parallel_coordinates(tune_result_df, color='mean_test_RMSLE')

### XGBoost

In [ ]:
from tune_sklearn import TuneSearchCV
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor

# Define a search space
parameters = {
    'estimator__n_estimators': [50, 100, 200],
    'estimator__learning_rate': [0.01, 0.1, 0.2],
    'estimator__min_child_weight': [5, 10, 15],
    'estimator__gamma': list(np.geomspace(1e-2, 1, 3)),
    'estimator__subsample': [0.6, 1.0],
    'estimator__colsample_bytree': [0.6, 1.0],
    'estimator__max_depth': [15, 20, 25],
    'estimator__objective': ['reg:squarederror'],
    'estimator__random_state': [config['random_state']]
}

# Define a Multi-output regressor
base_regr = XGBRegressor(verbosity=0)
regressor = MultiOutputRegressor(base_regr)

# Specify a hyper parameter tuning algorithm
tune_search = TuneSearchCV(
    regressor,
    parameters,
    search_optimization='hyperopt',
    n_trials=10,
    n_jobs=-1,
    scoring={'RMSLE':target_metric},
    cv=5,
    refit='RMSLE',
    verbose=1,
    random_state=config['random_state']
    )

# Run hyper parameter tuning
X = X_train
y = y_train_two_trans
tune_search.fit(X, y)

# Save the tuning results 
model_name = 'xgb'

## Save the optimal hyper parmater values
best_cv_df.loc[best_cv_df['model']==model_name, 'best_hyper_param'] = str(tune_search.best_params_)

## Save the CV results
cv_df = pd.DataFrame(tune_search.cv_results_)
cv_values = cv_df.loc[tune_search.best_index_, cv_df.columns.str.startswith('split')].values
best_cv_df.loc[best_cv_df['model']==model_name, 'RMSLE'] = cv_values[:5]

# Visualize the tuning results with parallel coordinate plot
tune_result_df = pd.concat([pd.DataFrame(tune_search.cv_results_['params']), cv_df.loc[:,cv_df.columns.str.startswith('mean')] ], axis=1)
import plotly.express as px
px.parallel_coordinates(tune_result_df, color='mean_test_RMSLE')

### Multi-layer perceptron

In [ ]:
import torch
from torch import nn
from skorch.regressor import NeuralNetRegressor
from skorch.callbacks import EarlyStopping
from skorch.callbacks import EpochScoring
from skorch.callbacks import Checkpoint
from tune_sklearn import TuneSearchCV
from sklearn.multioutput import MultiOutputRegressor

# Define the model structure 
# - Note that MLP does not need to implement the 'MultiOutputRegressor' function from sklearn 
# - By adjusting the number of output layer unit, we can get the Multi-outputs
class MLP(nn.Module):
    def __init__(self, num_inputs=X_train.shape[1], num_outputs=2, layer1=512, layer2=256, dropout1=0, dropout2=0):
        super(MLP, self).__init__()

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(num_inputs, layer1),
            nn.LeakyReLU(),
            nn.Dropout(dropout1),
            nn.Linear(layer1, layer2),
            nn.LeakyReLU(),
            nn.Dropout(dropout2),
            nn.Linear(layer2, num_outputs)
            )
    def forward(self, x):
        x = self.linear_relu_stack(x)
        return x  

def try_gpu(i=0): 
    return f'cuda:{i}' if torch.cuda.device_count() >= i + 1 else 'cpu'

# Set model configurations
# - We can set the custom scoring function for early stopping by using 'callbacks'
# - Set the scoring function as 'target_metric' which is the custom evaluation function we made above
mlp = NeuralNetRegressor(
    MLP(num_inputs=X_train.shape[1], num_outputs=2),
    optimizer=torch.optim.Adam,
    criterion=nn.MSELoss,
    iterator_train__shuffle=True,
    device=try_gpu(),
    verbose=0,
    callbacks=[EpochScoring(target_metric, lower_is_better=False, on_train=False, name='valid_neg_rmsle_custom'),
               EarlyStopping(monitor='valid_neg_rmsle_custom', patience=5,
                             threshold=1e-3, lower_is_better=False),
               Checkpoint(monitor='valid_neg_rmsle_custom_best')]
                          )

# Define a search space
parameters = {
    'lr': list(np.geomspace(1e-5, 1e-2, 4)),
    'module__layer1': [128, 256],
    'module__layer2': [128, 256],
    'module__dropout1': [0, 0.1],
    'module__dropout2': [0, 0.1],
    'optimizer__weight_decay': list(np.append(0, np.geomspace(1e-6, 1e-1, 6))),
    'max_epochs': [1000],
    'batch_size': [128],
    'callbacks__EarlyStopping__threshold': [1e-3, 1e-4]
    }

def use_gpu(device):
    return True if not device == 'cpu' else False 

# Specify the hyper parameter tuning algorithm
tune_search = TuneSearchCV(
    mlp,
    parameters,
    search_optimization='hyperopt',
    n_trials=15,
    n_jobs=-1,
    scoring={'RMSLE': target_metric},
    cv=5,
    refit='RMSLE',
    verbose=1,
    random_state=config['random_state']
    )

# Run hyper parameter tuning
X = X_train
y = y_train_two_trans
tune_search.fit(X, y)

# Save the tuning results 
model_name = 'mlp'

## Save the optimal hyper parmater values
best_cv_df.loc[best_cv_df['model']==model_name, 'best_hyper_param'] = str(tune_search.best_params_)

## Save the CV results
cv_df = pd.DataFrame(tune_search.cv_results_)
cv_values = cv_df.loc[tune_search.best_index_, cv_df.columns.str.startswith('split')].values
best_cv_df.loc[best_cv_df['model']==model_name, 'RMSLE'] = cv_values[:5]

# Visualize the tuning results with parallel coordinate plot
tune_result_df = pd.concat([pd.DataFrame(tune_search.cv_results_['params']), cv_df.loc[:,cv_df.columns.str.startswith('mean')] ], axis=1)
tune_result_df.rename({
    'callbacks__EarlyStopping__threshold':'Earlystoping_threshold',
    'optimizer__weight_decay': 'weight_decay'
    }, axis=1, inplace=True)
import plotly.express as px
px.parallel_coordinates(tune_result_df, color='mean_test_RMSLE')

### Factorization Machine

In [ ]:
def prepro_for_fm(X_train, X_test, bin_method='sturges'):
  n_train = X_train.shape[0]
  all = np.vstack((X_train, X_test))

  col_num_uniq = np.apply_along_axis(lambda x: len(np.unique(x)), 0,  all)
  remain_iidx = (col_num_uniq<=2)
  to_bin_iidx = (col_num_uniq>2)

  all_remain = all[:,remain_iidx]
  all_to_bin = all[:,to_bin_iidx]
  
  for iter in range(all_to_bin.shape[1]):
    bin_size = len(np.histogram(all_to_bin[:,iter], bins=bin_method)[0])
    all_to_bin[:,iter] = pd.cut(all_to_bin[:,iter], bins=bin_size, labels=False)

  all_to_bin_df = pd.DataFrame(all_to_bin).astype('object')
  all_to_bin_array = pd.get_dummies(all_to_bin_df, drop_first=False).to_numpy()

  all_array = np.hstack((all_to_bin_array, all_remain)).astype(np.int64)
  field_dims = all_array.shape[1]
  all_fm = np.vstack((np.apply_along_axis(lambda x: np.where(x==1), 1, all_array)))

  return all_fm[:n_train], all_fm[n_train:], field_dims


X_train_fm, X_test_fm, field_dims = prepro_for_fm(X_train, X_test, bin_method='sturges')

In [ ]:
import torch
from torch import nn
from skorch.regressor import NeuralNetRegressor
from skorch.callbacks import EarlyStopping
from skorch.callbacks import EpochScoring
from skorch.callbacks import Checkpoint
from tune_sklearn import TuneSearchCV
from sklearn.multioutput import MultiOutputRegressor

# Define the model structure 
# - Note that FM does not need to implement the 'MultiOutputRegressor' function from sklearn 
# - By adjusting the number of output layer unit, we can get the Multi-outputs
class FM(nn.Module):
    def __init__(self, num_inputs=field_dims, num_factors=20, output_dim=2):
        super(FM, self).__init__()
        self.output_dim = output_dim
        for i in range(output_dim):
          setattr(self, f'embedding_{i}', nn.Embedding(num_inputs, num_factors))
        self.fc = nn.Embedding(num_inputs, output_dim)
        self.bias = nn.Parameter(torch.zeros((output_dim,)))

    def forward(self, x):
        square_of_sum_list = []
        sum_of_square_list = []
        for i in range(self.output_dim):
          square_of_sum_list.append(torch.sum(getattr(self, f'embedding_{i}')(x), dim=1)**2)
          sum_of_square_list.append(torch.sum(getattr(self, f'embedding_{i}')(x)**2, dim=1))
        square_of_sum = torch.stack(square_of_sum_list, dim=1)
        sum_of_square = torch.stack(sum_of_square_list, dim=1)
        x = self.bias + self.fc(x).sum(dim=1) + 0.5 * (square_of_sum - sum_of_square).sum(dim=2)
        return x

def try_gpu(i=0): 
    return f'cuda:{i}' if torch.cuda.device_count() >= i + 1 else 'cpu'

# Set model configurations
# - We can set the custom scoring function for early stopping by using 'callbacks'
# - Set the scoring function as 'target_metric' which is the custom evaluation function we made above
fm = NeuralNetRegressor(
    FM(num_inputs=field_dims, output_dim=2),
    optimizer=torch.optim.Adam,
    criterion=nn.MSELoss,
    iterator_train__shuffle=True,
    device=try_gpu(),
    verbose=0,
    callbacks=[EpochScoring(target_metric, lower_is_better=False, on_train=False, name='valid_neg_rmsle_custom'),
               EarlyStopping(monitor='valid_neg_rmsle_custom', patience=5,
                             threshold=1e-4, lower_is_better=False),
               Checkpoint(monitor='valid_neg_rmsle_custom_best')]
                          )

# Define a search space
parameters = {
    'lr': [0.05, 0.1, 0.5],
    'module__num_factors': [10, 20, 100],
    'optimizer__weight_decay': [0.005, 0.01, 0.05, 0.1],
    'max_epochs': [1000],
    'batch_size': [128, 256],
    'callbacks__EarlyStopping__threshold': [1e-3]
    }


def use_gpu(device):
    return True if not device == 'cpu' else False 

# Specify the hyper parameter tuning algorithm
tune_search = TuneSearchCV(
    fm,
    parameters,
    search_optimization='hyperopt',
    n_trials=10,
    n_jobs=-1,
    scoring={'RMSLE': target_metric},
    cv=5,
    refit='RMSLE',
    verbose=1,
    random_state=config['random_state']
    )

# Run hyper parameter tuning
X = X_train_fm
y = y_train_two_trans
tune_search.fit(X, y)

# Save the tuning results 
model_name = 'fm'

## Save the optimal hyper parmater values
best_cv_df.loc[best_cv_df['model']==model_name, 'best_hyper_param'] = str(tune_search.best_params_)

## Save the CV results
cv_df = pd.DataFrame(tune_search.cv_results_)
cv_values = cv_df.loc[tune_search.best_index_, cv_df.columns.str.startswith('split')].values
best_cv_df.loc[best_cv_df['model']==model_name, 'RMSLE'] = cv_values[:5]

# Visualize the tuning results with parallel coordinate plot
tune_result_df = pd.concat([pd.DataFrame(tune_search.cv_results_['params']), cv_df.loc[:,cv_df.columns.str.startswith('mean')] ], axis=1)
tune_result_df.rename({
    'callbacks__EarlyStopping__threshold':'Earlystoping_threshold',
    'optimizer__weight_decay': 'weight_decay'
    }, axis=1, inplace=True)
tune_result_df = tune_result_df[~tune_result_df['mean_test_RMSLE'].isnull()]
import plotly.express as px
px.parallel_coordinates(tune_result_df, color='mean_test_RMSLE')

In [ ]:
import os 
save_path = '/content/{}/Result'.format(config['data_name'])
if not os.path.exists(save_path):
  os.makedirs(save_path)
file_path = os.path.join(save_path, 'best_cv_results.csv')
best_cv_df.to_csv(file_path, index=False)

## 2-2. Model comparison based on CV results with best hyper parameters

Compare the CV results (measured using the optimal hyper parameter values)

The figure below shows that the rf, xgb, mlp, and fm models show superior performance compared to the linear and svm models.



In [ ]:
fig = px.box(best_cv_df, x='model', y='RMSLE', color='model', width=600)
fig.show()

## 2-3. Model Combination

Although it is possible to select a final model among the models above, it has been observed that in many cases the combination of predicted values ​​from multiple models leads to improved prediction performance. ([Can multi-model combination really enhance the prediction skill of probabilistic ensemble forecasts?](https://rmets.onlinelibrary.wiley.com/doi/abs/10.1002/qj.210?casa_token=OwyF2RbEywAAAAAA:gahpwGRdOWzLXyafYQQt_voHOF8MedTBLd1SBv4vkdT3ZTLVoKZQj3zl-KbrhSkX5x8CndeCxwBoL_-S))

For regression problems, the final predicted values are derived by combining the predicted values in a 'proper way'.

This notebook uses the following two model combination methods.

1. Simple Average
2. Stacked Generalization (Stacking)


Model comparison needs to be done with single models (e.g., rf, xgb,...).
So model performance are measured by applying the same CV method as above.

### > Simple Average

The simple average method derives the final prediction value by 'averaging' the predicted values of multiple models.

The top 3 models (svm, rf, xgb) of the above CV results are selected as base estimators used for the combination of predicted values.

For example,
- Base Estimations
  - $f_{svm}(x)$ = 0.85
  - $f_{rf}(x)$ = 0.75
  - $f_{xgb}(x)$ = 0.80
- Final Estimation
  - $f_{average}(x)$  = 0.8 (= 0.85 + 0.75 + 0.80 + / 3)


In [ ]:
from sklearn.model_selection import KFold
from tqdm import notebook
from sklearn.metrics import mean_squared_error

def CV_ensemble(ensemble_name, ensemble_func, estimators, X_train, y_train, n_folds=5, shuffle=True, random_state=2022):
  kf = KFold(n_splits=5, random_state=random_state, shuffle=True)

  res_list = []
  for train_idx, valid_idx in notebook.tqdm(kf.split(X_train), total=kf.get_n_splits(), desc='Eval_CV'):
    X_train_train, X_valid = X_train[train_idx], X_train[valid_idx]
    y_train_train, y_valid = y_train[train_idx], y_train[valid_idx]

    ensemble_pred = ensemble_func(estimators, X_train_train, y_train_train, X_valid)
    neg_rmsle = neg_rmsle_custom(y_valid, ensemble_pred)

    res_list.append([ensemble_name, neg_rmsle])
  res_df = pd.DataFrame(np.vstack((res_list)))
  res_df.columns = ['model', 'RMSLE']
  return res_df

def average_reg_multi_output(estimators, X_train, y_train, X_test):
  preds = []
  for iter in range(len(estimators)):
    try:
      estimators[iter].module__num_factors
    except: # for other models
      estimators[iter].fit(X_train, y_train)
      preds.append([estimators[iter].predict(X_test)])
    else: # for factorization machine
      X_train_fm, X_test_fm, _ = prepro_for_fm(X_train, X_test)
      estimators[iter].fit(X_train_fm, y_train)
      preds.append([estimators[iter].predict(X_test_fm)])
    
  avg_pred = np.vstack((preds)).mean(axis=0)
  return avg_pred

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor

linear = MultiOutputRegressor(SGDRegressor()).set_params(**eval(best_cv_df.loc[best_cv_df['model']=='linear', 'best_hyper_param'].values[0]))
svm = MultiOutputRegressor(SGDRegressor()).set_params(**eval(best_cv_df.loc[best_cv_df['model']=='svm', 'best_hyper_param'].values[0]))
rf = MultiOutputRegressor(RandomForestRegressor()).set_params(**eval(best_cv_df.loc[best_cv_df['model']=='rf', 'best_hyper_param'].values[0]))
xgb = MultiOutputRegressor(XGBRegressor()).set_params(**eval(best_cv_df.loc[best_cv_df['model']=='xgb', 'best_hyper_param'].values[0]))
mlp = mlp.set_params(**eval(best_cv_df.loc[best_cv_df['model']=='mlp', 'best_hyper_param'].values[0]))
fm = fm.set_params(**eval(best_cv_df.loc[best_cv_df['model']=='fm', 'best_hyper_param'].values[0]))

estimators = [xgb, mlp, fm]
estimators_name = 'xgb_mlp_fm'
ensemble_name = 'average' + '_by_' + estimators_name

X = X_train
y = y_train_two_trans

res_df = CV_ensemble(ensemble_name, average_reg_multi_output, estimators, X, y, n_folds=5, shuffle=True, random_state=config['random_state'])
best_cv_df = best_cv_df.append(res_df).reset_index(drop=True)

Eval_CV:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption

/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption

/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption

/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption

/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memor

In [ ]:
fig = px.box(best_cv_df, x='model', y='RMSLE', color='model', width=600)
fig.show()

In [ ]:
import os 
save_path = '/content/{}/Result'.format(config['data_name'])
if not os.path.exists(save_path):
  os.makedirs(save_path)
file_path = os.path.join(save_path, 'best_cv_results.csv')
best_cv_df.to_csv(file_path, index=False)

### > Stacked generalization (Stacking)

In the [Stacked generalization](https://www.jair.org/index.php/jair/article/view/10228), the predicted values of base estimators are treated as the 'input data', and y (SalePrice) of each row is treated as the 'output variable'. 
The 'Meta Learner' is learned with these data and the predicted values of this model are derived as the final prediction values.

- The 'Meta Learner' can be any of the regression models. However, this notebook uses a linear regression model for simplicity.

- As input data for 'Meta Learner', predicted values for validation data in CV of base estimators are obtained.

- Trained meta-learner predicts the final predicted values for the test-set by using the predicted values of the baes estimators for the test-set as input data.

The procedure is as follows:
1. (Base estimators) Run CV on Train-set
2. (Meta Learner) Train on CV predictions (predicted values on validation data of CV) with corresponding y values
3. (Base estimators) Train on Train-set
4. (Base estimators) Predict on Test-set
5. (Meta Learner) Predict on predicted values from step 4.

<img align='top' src='https://drive.google.com/uc?export=view&id=1uDxSIIFt8rUJkuIwRYU4lALvOPqlXPG5' width='600' height='400'>


For example,
- Base Estimations
  - $f_{svm}(x)$ = 0.85
  - $f_{rf}(x)$ = 0.75
  - $f_{xgb}(x)$ = 0.80
- Meta Learner (linear regression)
  - Parameter
    - intercept = 0.1
    - coefficient = [0.3, 0.1, 0.6]
  - $f_{stack}(x) = 0.795 = -0.1 + 0.4*0.85 + 0.1*0.75 + 0.6*0.80$

In [ ]:
from sklearn.model_selection import KFold
from tqdm import notebook

def stack_reg_multi_output(estimators, X_train, y_train, X_test, n_folds=5, shuffle=True, random_state=2022):
  num_estimators = len(estimators)-1
  num_output = y_train.shape[1]

  final_estimator = estimators[-1]
  kf = KFold(n_splits=n_folds, random_state=random_state, shuffle=shuffle)
  preds = []
  y_valid_list = []
  # Get CV predictions
  for train_idx, valid_idx in notebook.tqdm(kf.split(X_train), total=kf.get_n_splits(), desc='Stack_CV'):
    X_train_train, X_valid = X_train[train_idx], X_train[valid_idx]
    y_train_train, y_valid = y_train[train_idx], y_train[valid_idx]
    
    valid_preds = []
    for iter in range(num_estimators):
      try:
        estimators[iter].module__num_factors
      except: # for other models
        estimators[iter].fit(X_train_train, y_train_train)
        valid_preds.append([estimators[iter].predict(X_valid)])
      else: # for factorization machine
        X_train_train_fm, X_valid_fm, _ = prepro_for_fm(X_train_train, X_valid)
        estimators[iter].fit(X_train_train_fm, y_train_train)
        valid_preds.append([estimators[iter].predict(X_valid_fm)])
    
    preds.append(np.hstack((np.vstack((valid_preds)))))
    y_valid_list.append(y_valid)

  cv_preds = np.vstack((preds))
  cv_y = np.vstack((y_valid_list))
  
  # Get test predictions
  test_preds =[]
  for iter in range(num_estimators):
    try:
      estimators[iter].module__num_factors
    except: # for other models
      estimators[iter].fit(X_train, y_train)
      test_preds.append([estimators[iter].predict(X_test)])
    else: # for factorization machine
      X_train_fm, X_test_fm, _ = prepro_for_fm(X_train, X_test)
      estimators[iter].fit(X_train_fm, y_train)
      test_preds.append([estimators[iter].predict(X_test_fm)])

  test_preds_mat = np.hstack((np.vstack((test_preds))))

  # Fit the final estimator on prediction values
  final_estimator.fit(cv_preds, cv_y)
  print('Training RMSLE: {}'.format(target_metric(final_estimator, cv_preds, cv_y)))
  for iter in range(num_output):
    print('Estimated coefficients of model {}: {} \n intercept: {}'.format(iter, final_estimator.estimators_[iter].coef_, 
                                                                           final_estimator.estimators_[iter].intercept_))
  test_ensemble_pred = final_estimator.predict(test_preds_mat)
  return test_ensemble_pred

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)


linear = MultiOutputRegressor(SGDRegressor()).set_params(**eval(best_cv_df.loc[best_cv_df['model']=='linear', 'best_hyper_param'].values[0]))
svm = MultiOutputRegressor(SGDRegressor()).set_params(**eval(best_cv_df.loc[best_cv_df['model']=='svm', 'best_hyper_param'].values[0]))
rf = MultiOutputRegressor(RandomForestRegressor()).set_params(**eval(best_cv_df.loc[best_cv_df['model']=='rf', 'best_hyper_param'].values[0]))
xgb = MultiOutputRegressor(XGBRegressor()).set_params(**eval(best_cv_df.loc[best_cv_df['model']=='xgb', 'best_hyper_param'].values[0]))
mlp = mlp.set_params(**eval(best_cv_df.loc[best_cv_df['model']=='mlp', 'best_hyper_param'].values[0]))
fm = fm.set_params(**eval(best_cv_df.loc[best_cv_df['model']=='fm', 'best_hyper_param'].values[0]))

reg = MultiOutputRegressor(LinearRegression())

estimators = [xgb, mlp, fm, reg]

estimators_name = 'xgb_mlp_fm'
ensemble_name = 'stack_linear' + '_by_' + estimators_name
ensemble_func = stack_reg_multi_output
X = X_train
y = y_train_two_trans

res_df = CV_ensemble(ensemble_name, ensemble_func, estimators, X, y, n_folds=5, shuffle=True, random_state=config['random_state'])
best_cv_df = best_cv_df.append(res_df).reset_index(drop=True)

Eval_CV:   0%|          | 0/5 [00:00<?, ?it/s]

Stack_CV:   0%|          | 0/5 [00:00<?, ?it/s]

Training RMSLE: -0.2964572012424469
Estimated coefficients of model 0: [ 0.53238136  0.19314793  0.31630805 -0.16032687  0.15278806 -0.01874641] 
 intercept: -0.08037352561950684
Estimated coefficients of model 1: [ 0.04813029  0.6523862  -0.04985321  0.26961184  0.006147    0.09395757] 
 intercept: -0.1080470085144043


Stack_CV:   0%|          | 0/5 [00:00<?, ?it/s]

Training RMSLE: -0.28961867094039917
Estimated coefficients of model 0: [ 0.5369271   0.14672841  0.36700183 -0.12617753  0.0938362  -0.00689979] 
 intercept: -0.08205962181091309
Estimated coefficients of model 1: [ 5.5970058e-02  6.5895569e-01 -5.4127932e-02  3.3023480e-01
  2.7596951e-05  2.6351362e-02] 
 intercept: -0.09461784362792969


...

Training RMSLE: -0.29158246517181396
Estimated coefficients of model 0: [ 0.5475921   0.20013669  0.32261527 -0.13552207  0.12980449 -0.04952844] 
 intercept: -0.07906723022460938
Estimated coefficients of model 1: [ 0.05557633  0.69089353 -0.0555326   0.2707539   0.00183985  0.05626829] 
 intercept: -0.10709047317504883


Stack_CV:   0%|          | 0/5 [00:00<?, ?it/s]

Training RMSLE: -0.2837909460067749
Estimated coefficients of model 0: [ 0.5535143   0.1984666   0.30487505 -0.14339218  0.14082031 -0.03598769] 
 intercept: -0.10804104804992676
Estimated coefficients of model 1: [ 0.07847983  0.65196705 -0.07782093  0.33635145 -0.0011839   0.03354809] 
 intercept: -0.10647153854370117


Stack_CV:   0%|          | 0/5 [00:00<?, ?it/s]

Training RMSLE: -0.28825366497039795
Estimated coefficients of model 0: [ 0.50756836  0.23763826  0.36064613 -0.18130116  0.12830085 -0.03905784] 
 intercept: -0.08271169662475586
Estimated coefficients of model 1: [ 0.05210269  0.71606535 -0.05899441  0.30389518  0.00903875 -0.00373256] 
 intercept: -0.10210704803466797


In [ ]:
import os 
save_path = '/content/{}/Result'.format(config['data_name'])
if not os.path.exists(save_path):
  os.makedirs(save_path)
file_path = os.path.join(save_path, 'best_cv_results.csv')
best_cv_df.to_csv(file_path, index=False)

## 2-4. Model Comparison based on CV results including model combination methods

From the figure below, 'xgb' shows the best performance among single models.
Among the model combination methodologies, it can be seen that the 'stack_ridge_by_rf_xgb_mlp_fm' method shows the best performance.

In [ ]:
import plotly.express as px
fig = px.box(best_cv_df, x='model', y='RMSLE', color='model', width=1000)
fig.show()

# 3. Make a prediction with the best model

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

linear = MultiOutputRegressor(SGDRegressor()).set_params(**eval(best_cv_df.loc[best_cv_df['model']=='linear', 'best_hyper_param'].values[0]))
svm = MultiOutputRegressor(SGDRegressor()).set_params(**eval(best_cv_df.loc[best_cv_df['model']=='svm', 'best_hyper_param'].values[0]))
rf = MultiOutputRegressor(RandomForestRegressor()).set_params(**eval(best_cv_df.loc[best_cv_df['model']=='rf', 'best_hyper_param'].values[0]))
xgb = MultiOutputRegressor(XGBRegressor()).set_params(**eval(best_cv_df.loc[best_cv_df['model']=='xgb', 'best_hyper_param'].values[0]))
mlp = mlp.set_params(**eval(best_cv_df.loc[best_cv_df['model']=='mlp', 'best_hyper_param'].values[0]))
fm = fm.set_params(**eval(best_cv_df.loc[best_cv_df['model']=='fm', 'best_hyper_param'].values[0]))

estimators = [xgb, mlp, fm, reg]
estimators_name = 'xgb_mlp_fm'


reg = MultiOutputRegressor(LinearRegression())

ensemble_func = stack_reg_multi_output
ensemble_name = 'stack_linear_by_{}'.format(estimators_name)


X = X_train
y = y_train_two_trans

pred_two = ensemble_func(estimators, X, y, X_test, n_folds=5, shuffle=True, random_state=config['random_state'])
pred_trans = np.expm1(pred_two).sum(axis=1)
res_df = pd.DataFrame({'datetime': test['datetime'], 'count': pred_trans})

res_df.to_csv('{}.csv'.format(ensemble_name), index=False)

Stack_CV:   0%|          | 0/5 [00:00<?, ?it/s]

Training RMSLE: -0.28444793820381165
Estimated coefficients of model 0: [ 0.5413572   0.1669846   0.30031532 -0.10824166  0.14802362 -0.03728894] 
 intercept: -0.0807044506072998
Estimated coefficients of model 1: [ 0.04289532  0.68189573 -0.03148285  0.22348021 -0.01086798  0.11121519] 
 intercept: -0.09993982315063477
